In [25]:
from openai import OpenAI
import pandas as pd
from dataset_manager import NewsDataLoader, PriceDataLoader
import datetime

In [26]:
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)

In [27]:
price_data = PriceDataLoader()
news_data = NewsDataLoader()

In [28]:
tickers = sorted(tuple(set(price_data.get_available_tickers()).intersection(set(news_data.get_available_tickers()))))

In [29]:
ticker = tickers[0]
stock_target = ticker

factor_k = 5
price_k = 5

dates = sorted(set(price_data.get_available_dates(ticker, price_k)).intersection(set(news_data.get_available_dates(ticker))))
target_date = dates[1]

In [30]:
target_date

Timestamp('2018-01-10 00:00:00')

In [31]:
news_str = news_data.get_news_by_date(ticker, target_date)
price_movements = price_data.get_price_movements(ticker, target_date, price_k)

In [32]:
price_movements

[{'date': Timestamp('2018-01-04 00:00:00'), 'rise': True},
 {'date': Timestamp('2018-01-05 00:00:00'), 'rise': True},
 {'date': Timestamp('2018-01-08 00:00:00'), 'rise': False},
 {'date': Timestamp('2018-01-09 00:00:00'), 'rise': False},
 {'date': Timestamp('2018-01-10 00:00:00'), 'rise': False}]

In [33]:
# Format price movement data
historical_price_str_format = "On {date}, the stock price of {stock_target} {risefall}.\n"
today_price_str_format = "On {date}, the stock price of {stock_target} will ____.\n"
price_str = ""
for move in price_movements[:-1]:
    price_str += historical_price_str_format.format(date=move['date'].strftime('%Y-%m-%d'), stock_target=stock_target, risefall="rose" if move['rise'] else "fell")
price_str_assistant = today_price_str_format.format(date=target_date.strftime('%Y-%m-%d'), stock_target=stock_target)

# Extract 
factor_extraction = client.chat.completions.create(
  model="meta-llama/Llama-3.2-3B-Instruct",
  temperature=0,
  messages=[
    {"role": "system", "content": f"Please extract the top {factor_k} factors that may affect the stock price of {stock_target} from the following news."},
    {"role": "user", "content": news_str}
  ]
)

factor_str = factor_extraction.choices[0].message.content

answer_extraction = client.chat.completions.create(
  model="meta-llama/Llama-3.2-3B-Instruct",
  temperature=0,
  messages=[
    {"role": "system", "content": "Based on the following information, please judge the direction of the stock price from rise/fall, fill in the blank and give reasons."},
    {"role": "system", "content": f"These are the main factors that may affect this stock’s price recently: {factor_str}."},
    {"role": "system", "content": price_str},
    {"role": "assistant", "content": price_str_assistant},
  ]
)

answer = answer_extraction.choices[0].message.content

In [35]:
print(answer.split('\n'))

On 2018-01-10, the stock price of AAPL will likely **fall**.  **Reasons:**  1. **Recent Trend**: The stock has shown a downward trend over the past two trading days (2018-01-08 and 2018-01-09), indicating a potential continuation of this trend unless there are significant positive developments.  2. **iPhone Battery Slowdown Controversy**: The ongoing controversy regarding the iPhone battery slowdown could continue to weigh on investor sentiment, as it raises concerns about consumer trust and potential regulatory actions.  3. **China Data Compliance**: The decision to hand over cloud operations in China to a state-owned partner might raise concerns about Apple's control over its data and operations in a crucial market, potentially affecting investor confidence.  4. **Market Sentiment**: The combination of these factors, along with the broader market sentiment, could contribute to continued pressure on the stock price.  While positive developments, such as the strategic acquisition of Sh